In [ ]:
import requests
import pandas as pd

GITHUB_TOKEN = ''

user_url = 'https://api.github.com/search/users'
params = {
    'q': 'location:"Shanghai" followers:>200',
    'sort': 'followers',
    'order': 'desc',
    'per_page': 100,
    'page': 1
}

headers = {
    'Authorization': f'token {GITHUB_TOKEN}'
}

user_data = []
while True:
    response = requests.get(user_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        users = data.get('items', [])

        if not users:
            break
        for user in users:
            user_info = {
                'login': user.get('login'),
            }
            user_data.append(user_info)
        params['page'] += 1
    else:
        print(f"Error: {response.status_code} - {response.text}")
        break

for user in user_data:
    user_details_response = requests.get(f"https://api.github.com/users/{user['login']}", headers=headers)

    if user_details_response.status_code == 200:
        user_details = user_details_response.json()
        user['name'] = user_details.get('name')
        user['company'] = user_details.get('company').strip().lstrip('@').upper() if user_details.get('company') else ''
        user['location'] = user_details.get('location')
        user['email'] = user_details.get('email')
        user['hireable'] = user_details.get('hireable')
        user['bio'] = user_details.get('bio')
        user['public_repos'] = user_details.get('public_repos')
        user['followers'] = user_details.get('followers')
        user['following'] = user_details.get('following')
        user['created_at'] = user_details.get('created_at')
    else:
        print(f"Error fetching details for {user['login']}: {user_details_response.status_code}")

df = pd.DataFrame(user_data)
df.to_csv('users.csv', index=False)

print("Data saved to users.csv")

from google.colab import files
files.download('users.csv')


Data saved to users.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
GITHUB_TOKEN = ''

user_url = 'https://api.github.com/search/users'
params = {
    'q': 'location:"Shanghai" followers:>200',
    'sort': 'followers',
    'order': 'desc',
    'per_page': 100,
    'page': 1
}

headers = {
    'Authorization': f'token {GITHUB_TOKEN}'
}

user_data = []
while True:
    response = requests.get(user_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        users = data.get('items', [])

        if not users:
            break

        for user in users:
            user_info = {'login': user.get('login')}
            user_data.append(user_info)

        params['page'] += 1
    else:
        print(f"Error fetching users: {response.status_code}")
        break

for user in user_data:
    user_details_response = requests.get(f"https://api.github.com/users/{user['login']}", headers=headers)

    if user_details_response.status_code == 200:
        user_details = user_details_response.json()
        user['name'] = user_details.get('name', '')
        user['company'] = user_details.get('company', '').strip().lstrip('@').upper() if user_details.get('company') else ''
        user['location'] = user_details.get('location', 'Shanghai')
        user['email'] = user_details.get('email', '')
        user['hireable'] = user_details.get('hireable', False)
        user['bio'] = user_details.get('bio', '')
        user['public_repos'] = user_details.get('public_repos', 0)
        user['followers'] = user_details.get('followers', 0)
        user['following'] = user_details.get('following', 0)
        user['created_at'] = user_details.get('created_at', '')
    else:
        print(f"Error fetching details for {user['login']}: {user_details_response.status_code}")

users_df = pd.DataFrame(user_data)
users_df.to_csv('users.csv', index=False)

repositories_data = []

for user in user_data:
    login = user['login']
    repos_url = f"https://api.github.com/users/{login}/repos?sort=updated&per_page=100"

    page = 1
    while True:
        response = requests.get(f"{repos_url}&page={page}", headers=headers)

        if response.status_code == 200:
            repos = response.json()
            if not repos:
                break

            for repo in repos:
                repo_info = {
                    'login': login,
                    'full_name': repo.get('full_name'),
                    'created_at': repo.get('created_at'),
                    'stargazers_count': repo.get('stargazers_count', 0),
                    'watchers_count': repo.get('watchers_count', 0),
                    'language': repo.get('language', ''),
                    'has_projects': repo.get('has_projects', False),
                    'has_wiki': repo.get('has_wiki', False),
                    'license_name': repo.get('license')['name'] if repo.get('license') else ''
                }
                repositories_data.append(repo_info)

                if len(repositories_data) % 500 == 0:
                    break

            page += 1

        else:
            print(f"Error fetching repositories for {login}: {response.status_code}")
            break

repos_df = pd.DataFrame(repositories_data)
repos_df.to_csv('repositories.csv', index=False)

print("Data saved to users.csv and repositories.csv")

from google.colab import files
files.download('repositories.csv')


Data saved to users.csv and repositories.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from collections import Counter

repos_df = pd.read_csv('repositories.csv')
license_names = repos_df['license_name'].dropna().tolist()
license_counts = Counter(license_names)
most_common_licenses = license_counts.most_common(3)
top_3_licenses = [license[0] for license in most_common_licenses]
top_3_licenses_str = ', '.join(top_3_licenses)
print("Top 3 licenses:", top_3_licenses_str)


Top 3 licenses: MIT License, Apache License 2.0, Other


In [ ]:
import pandas as pd

repos_df = pd.read_csv('repositories.csv')
average_stars_per_language = repos_df.groupby('language')['stargazers_count'].mean()
highest_avg_language = average_stars_per_language.idxmax()
highest_avg_stars = average_stars_per_language.max()
print(f"The language with the highest average number of stars per repository is '{highest_avg_language}' with an average of {highest_avg_stars:.2f} stars.")


The language with the highest average number of stars per repository is 'Rich Text Format' with an average of 1494.14 stars.


In [ ]:
import pandas as pd


users_df = pd.read_csv('users.csv')
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])
top_5_users = users_df.nlargest(5, 'leader_strength')
top_5_logins = ', '.join(top_5_users['login'].tolist())
print("Top 5 users by leader_strength:", top_5_logins)


Top 5 users by leader_strength: ruanyf, peng-zhihui, espressif, vnpy, bilibili


In [ ]:
import pandas as pd

users_df = pd.read_csv('users.csv')
correlation = users_df['followers'].corr(users_df['public_repos'])
print(f"Correlation between followers and public repositories: {correlation:.3f}")

Correlation between followers and public repositories: -0.005


In [ ]:
import pandas as pd
from scipy.stats import linregress

users_df = pd.read_csv('users.csv')
slope, intercept, r_value, p_value, std_err = linregress(users_df['public_repos'], users_df['followers'])
print(f"Regression slope of followers on public repositories: {slope:.3f}")


Regression slope of followers on public repositories: -0.054


In [ ]:
import pandas as pd

repos_df = pd.read_csv('repositories.csv')

repos_df['has_projects'] = repos_df['has_projects'].astype(int)
repos_df['has_wiki'] = repos_df['has_wiki'].astype(int)
correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])
print(f"Correlation between projects and wiki enabled: {correlation:.3f}")


Correlation between projects and wiki enabled: 0.280


In [ ]:
import pandas as pd

repos_df = pd.read_csv('repositories.csv')
repos_df['has_projects'] = repos_df['has_projects'].astype(int)
repos_df['has_wiki'] = repos_df['has_wiki'].astype(int)
correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])
print(f"Correlation between projects and wiki enabled: {correlation:.3f}")


Correlation between projects and wiki enabled: 0.280


In [ ]:
import pandas as pd

repos_df = pd.read_csv('repositories.csv')
print(repos_df.head())
repos_df['has_projects'] = repos_df['has_projects'].replace({True: 1, False: 0})
repos_df['has_wiki'] = repos_df['has_wiki'].replace({True: 1, False: 0})
correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])
print(f"Correlation between projects and wiki enabled: {correlation:.3f}")


         login                       full_name            created_at  \
0  peng-zhihui     peng-zhihui/agibot_x1_train  2024-10-24T05:25:33Z   
1  peng-zhihui         peng-zhihui/Dummy-Robot  2021-10-07T11:42:58Z   
2  peng-zhihui           peng-zhihui/HoloCubic  2021-01-07T03:24:11Z   
3  peng-zhihui         peng-zhihui/ElectronBot  2022-03-11T09:24:44Z   
4  peng-zhihui  peng-zhihui/agibot_x1_hardware  2024-10-24T08:57:19Z   

   stargazers_count  watchers_count language  has_projects  has_wiki  \
0                29              29      NaN          True     False   
1             12220           12220        C          True      True   
2              6073            6073        C          True      True   
3              7972            7972        C          True      True   
4                57              57      NaN          True     False   

                      license_name  
0                              NaN  
1                              NaN  
2  GNU General Public L

<ipython-input-32-85c3220b7f78>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  repos_df['has_projects'] = repos_df['has_projects'].replace({True: 1, False: 0})
<ipython-input-32-85c3220b7f78>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  repos_df['has_wiki'] = repos_df['has_wiki'].replace({True: 1, False: 0})


In [ ]:
import pandas as pd

users_df = pd.read_csv('users.csv')
average_following_hireable = users_df[users_df['hireable'] == True]['following'].mean()
average_following_non_hireable = users_df[users_df['hireable'] == False]['following'].mean()
difference = average_following_hireable - average_following_non_hireable
print(f"Average following per hireable user minus non-hireable user: {difference:.3f}")


Average following per hireable user minus non-hireable user: nan


In [ ]:
import pandas as pd

repos_df = pd.read_csv('repositories.csv')
repos_df['created_at'] = pd.to_datetime(repos_df['created_at'])
repos_df['weekday'] = repos_df['created_at'].dt.weekday
weekend_repos = repos_df[(repos_df['weekday'] == 5) | (repos_df['weekday'] == 6)]
user_repo_counts = weekend_repos['login'].value_counts()
top_5_users = user_repo_counts.head(5).index.tolist()
top_5_logins = ', '.join(top_5_users)
print("Top 5 users who created the most repositories on weekends:", top_5_logins)


Top 5 users who created the most repositories on weekends: Hengle, j5s, Ewenwan, HiWong, shadowcz007


In [ ]:
import pandas as pd
users_df = pd.read_csv('users.csv')
hireable_with_email = users_df[users_df['hireable'] == True]['email'].notnull().mean()
non_hireable_with_email = users_df[users_df['hireable'] == False]['email'].notnull().mean()
print(f"Fraction of users with email (hireable) minus fraction of users with email (non-hireable): {email_fraction_difference:.3f}")


Fraction of users with email (hireable) minus fraction of users with email (non-hireable): nan


In [ ]:
import pandas as pd
from collections import Counter

users_df = pd.read_csv('users.csv')

surnames = []
for name in users_df['name'].dropna():
    trimmed_name = name.strip()
    surname = trimmed_name.split()[-1]
    surnames.append(surname)

surname_counts = Counter(surnames)
most_common_count = max(surname_counts.values())
most_common_surnames = [surname for surname, count in surname_counts.items() if count == most_common_count]
most_common_surnames.sort()
print("Most common surname(s):", ', '.join(most_common_surnames))


Most common surname(s): Zhang


In [ ]:
import pandas as pd
import statsmodels.api as sm

users_df = pd.read_csv('users.csv')
users_df['bio_word_count'] = users_df['bio'].dropna().apply(lambda x: len(str(x).split()))
filtered_df = users_df[users_df['bio'].notnull()]
X = filtered_df['bio_word_count']
y = filtered_df['followers']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
slope = model.params['bio_word_count']
print(f"Regression slope of followers on bio word count: {slope:.3f}")


Regression slope of followers on bio word count: -42.233


In [ ]:
import pandas as pd

users_df = pd.read_csv('users.csv')
hireable_with_email = users_df[users_df['hireable'] == True]['email'].notnull().mean()
non_hireable_with_email = users_df[users_df['hireable'] == " "]['email'].notnull().mean()
email_fraction_difference = hireable_with_email - non_hireable_with_email
print(f"Fraction of users with email (hireable) minus fraction of users with email (non-hireable): {email_fraction_difference:.3f}")


Fraction of users with email (hireable) minus fraction of users with email (non-hireable): nan


In [ ]:
import pandas as pd

users_df = pd.read_csv('users.csv')
hireable_users = users_df[users_df['hireable'] == True]
hireable_with_email_fraction = hireable_users['email'].notnull().mean() if not hireable_users.empty else 0
non_hireable_users = users_df[users_df['hireable'].isnull()]
non_hireable_with_email_fraction = non_hireable_users['email'].notnull().mean() if not non_hireable_users.empty else 0
email_fraction_difference = hireable_with_email_fraction - non_hireable_with_email_fraction
print(f"Fraction of users with email (hireable) minus fraction of users with email (non-hireable): {email_fraction_difference:.3f}")


Fraction of users with email (hireable) minus fraction of users with email (non-hireable): 0.074


In [ ]:
import pandas as pd

users_df = pd.read_csv('users.csv')
hireable_users = users_df[users_df['hireable'] == True]
avg_following_hireable = hireable_users['following'].mean() if not hireable_users.empty else 0
non_hireable_users = users_df[users_df['hireable'].isnull()]
avg_following_non_hireable = non_hireable_users['following'].mean() if not non_hireable_users.empty else 0
following_difference = avg_following_hireable - avg_following_non_hireable
print(f"Average following (hireable) minus average following (non-hireable): {following_difference:.3f}")


Average following (hireable) minus average following (non-hireable): 142.854
